In [16]:
import datetime
import time
import urllib2
from random import randint

from careerjet_api_client import CareerjetAPIClient
import pymysql as mdb
from bs4 import BeautifulSoup

In [ ]:
# Create table for job descriptions

In [17]:
# Query jobs for their URL and id
con = mdb.connect('localhost', 'root', '', 'job_profiler', autocommit=True,
                 charset='utf8', use_unicode=True)
cur = con.cursor()

cur.execute("""SELECT pid, url FROM cj_jobs WHERE first_seen > '2016-01-04' AND q_location NOT IN
            ('san francisco', 'san jose') AND q_term IN (
                  'data scientist', 
                  'cognitive',
                  'user research',
                  'user researcher',
                  'user experience',
                  'ux',
                  'human computer interaction',
                  'HCI', 
                  'human factor engineer', 
                  'human factor engineering',
                  'usability',
                  'quantitative researcher',
                  'quantitative research',
                  'human factors')""")

origin_rows = cur.fetchall()

In [18]:
len(origin_rows)

939

In [19]:
import warnings
warnings.filterwarnings("error")

def removeNonAscii(s):
    return "".join(i for i in s if ord(i)<128)

In [ ]:
# affid saved in a text file
id_file = open('resources/careerjet_affid.txt')
my_id = id_file.readline()
my_id = my_id.strip()

# Append this to the URL
affid_q = '?affid='+my_id

q_base = "INSERT INTO cj_job_descriptions (id, html_text, job_text, url) \
     VALUES \
    (%s,%s,%s,%s)"

count = 0

print "Processing %i rows..." % len(origin_rows)

for pid, url in origin_rows:
    
    # Check that ID is not already in table
    cur.execute("SELECT * FROM cj_job_descriptions WHERE id = %s" % pid)
    rows = cur.fetchall()
    # If so, skip this job
    if rows:
        #print "Record exists"
        continue
    
    # "API courtesy"
    time.sleep(randint(1,2))
    
    try:
        # Read the URL
        page = urllib2.urlopen(url).read() 
    except Exception:
        print "Skipping"
        continue
        
    # Split on URL key
    page = page.split("URL=")
    # target URL is at beginning of second element in list
    if len(page) == 1:
        continue
    page = page[1].split('">\n<')[0]
    # Append affid to get a careerjet URL (otherwise redirects)
    reurl = urllib2.urlopen(page+affid_q).read()
    soup = BeautifulSoup(reurl, "lxml")
    result = soup.find("div", {"class": "advertise_compact"})
    
    # Scrub, take care of unicode
    
    #clean_html = str(result)
    clean_html = removeNonAscii(result.text.encode('utf8', 'ignore'))
    clean_html = clean_html.replace('"', r'\"')
    clean_html = clean_html.replace("'", r"\'")
    #clean_text = result.text.encode('utf-8')   # properly encode unicode
    clean_text = removeNonAscii(result.text.encode('utf8', 'ignore'))
    clean_text = clean_text.replace('"', r'\"')
    clean_text = clean_text.replace("'", r"\'")
    
    # Insert into table
    q_val = (pid, clean_html, clean_text, str(page+affid_q))
    cur.execute(q_base, q_val)
    
    count += 1
    
    print count, (pid, clean_html[:20], clean_text[:20], str(page+affid_q))
    
    if count%10 == 0:
        print "Waiting 5 secs..."
        time.sleep(1)
        #print "Resuming"
    if count%50 == 0:
        print "Stored %i out of %i..." % (count, len(origin_rows))

print "Done", count

In [6]:
url


u'http://jobviewtrack.com/en-us/job-4c1c415e4e0d0a00531d53176a121c4716431d0d07074358492f5d0c0e1c4a15430c030800021149171b6b03424a4f404c0700054e1a451701021d4716431d0d07074358492c1a515c5d/c6da6aa628d5b901a2baad532e6aee98.html?affid=2dba63bec709301650048b13f60d344d'